In [21]:
%load_ext google.cloud.bigquery
import pandas as pd
#import plotly.express as px
#import plotly.graph_objects as go

The google.cloud.bigquery extension is already loaded. To reload it, use:
  %reload_ext google.cloud.bigquery


This will select data from BQ into a dataframe

In [ ]:
%%bigquery prices
SELECT * FROM sell_prices.sell_prices where store_id = 'CA_1';

In [ ]:
prices.head()

In [ ]:
%%bigquery calendar
SELECT * FROM `kaggle-covid.calendar.calendar`;


In [ ]:
calendar

In [ ]:
%%bigquery item_013
-- select one item in one store
select 
*
from
sales_train_evaluation.sales_train_evaluation a
where a.store_id = 'CA_1' and a.item_id = 'HOBBIES_1_013';

In [ ]:
item_013

In [ ]:
%%bigquery store_CA_1
-- select all items from one store
select 
id, a.item_id, dept_id,cat_id, a.store_id, state_id, d_1, d_1941
from
sales_train_evaluation.sales_train_evaluation a
where a.store_id = 'CA_1';

In [14]:
store_CA_1

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,1
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,0,0
...,...,...,...,...,...,...,...,...
3044,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,5,0
3045,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,12,45
3046,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,33,21
3047,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,42,43


Pivot all the date columns into rows to get something more amenable for time series prediction 

In [15]:
df = pd.melt(
    store_CA_1,
    id_vars=[
        'id',
        'item_id',
        'dept_id',
        'cat_id',
        'store_id',
        'state_id'
    ],
    var_name='d',
    value_name='items_sold'
).dropna()

In [16]:
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
6093,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,d_1941,0
6094,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,d_1941,45
6095,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,d_1941,21
6096,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1941,43


Join with calendar dataframe

In [17]:
df = pd.merge(df, calendar, on='d', how='left')
df

,id,item_id,dept_id,cat_id,store_id,state_id,d,items_sold,date,wm_yr_wk,...,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
3,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
4,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_007,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,...,1,1,2011,None,None,None,None,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6093,FOODS_3_319_CA_1_evaluation,FOODS_3_319,FOODS_3,FOODS,CA_1,CA,d_1941,0,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0
6094,FOODS_3_681_CA_1_evaluation,FOODS_3_681,FOODS_3,FOODS,CA_1,CA,d_1941,45,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0
6095,FOODS_3_080_CA_1_evaluation,FOODS_3_080,FOODS_3,FOODS,CA_1,CA,d_1941,21,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0
6096,FOODS_3_586_CA_1_evaluation,FOODS_3_586,FOODS_3,FOODS,CA_1,CA,d_1941,43,2016-05-22,11617,...,2,5,2016,None,None,None,None,0,0,0


Now join with item prices for each item within store for that date (wm_yrwk).

In [18]:
df = pd.merge(
    df,
    prices,
    on=['store_id','item_id','wm_yr_wk'],
    how='left') 

TypeError: Can only merge Series or DataFrame objects, a <class 'google.cloud.bigquery.job.QueryJob'> was passed

In [ ]:
df.describe

In [ ]:
df.head()

In [ ]:
df.index

In [ ]:
df.loc[df['wm_yr_wk'] == 11101]

In [ ]:
def plot_values(df, from_date=None, target=None):
    fig = px.bar(df.loc[df['date'] >= from_date], x='date', \
             y='items_sold', color="items_sold", width=800, height=400, color_continuous_scale=px.colors.sequential.BuGn)
    fig.show()
    
plot_values(df, '2011-01-29')